# DT Tag-and-Probe (TnP) analysis

This notebook features a simplified version of the Tag-and-Probe analysis of the CMS Drift Tubes (DT) for detector performance studies.

In [1]:
distributed = False ## Default True - Enable DASK parallelisation
MT = True ## Multi-threading - Disabled by default if distributed is True

if distributed:
    sched_port = 21901 ## Change this from the DASK cluster information panel

### Dask cluster configuration

The following part configures the Dask cluster, that we launched previously in the JupyterLab interface.

In [2]:
if distributed:  
    from dask.distributed import Client
    
    client = Client("localhost:" + str(sched_port))

    client

In [3]:
#client.restart() #Execute this only to restart the workers (to relaunch the notebook, for example#

Upload the CMS X509 proxy (created using VOMS) in the Dask workers

In [4]:
if distributed:
    from distributed.diagnostics.plugin import UploadFile
    
    client.register_plugin(UploadFile("./proxy"))

Set the proxy environmental variables in the Dask workers

In [5]:
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'proxy'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR") 

In [6]:
if distributed:
    client.run(set_proxy)

### Import modules and local X509 proxy configuration

Import necessary libraries, setup the plotting tools, and configure the local X509 environment

In [7]:
import ROOT
import os
import json

from python import TnPTools
import configparser

#Enable multi-threading - Not possible if using RDF::Filter (nEvents != -1)
if distributed != True and MT == True:
    print("Going Multi-threaded")
    ROOT.ROOT.EnableImplicitMT()

num_stations = 4
num_wheels = 5
num_sectors = 14

# Loading PROXY locally
os.environ['X509_USER_PROXY'] = "proxy"
os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"

Going Multi-threaded


## Drift Tubes TnP analysis

The following cells perform the DT Tag-and-Probe analysis ([CMS-DP-2023-049](https://cds.cern.ch/record/2868786?ln=en))

First, we load the .INI file containing all the analysis variables initialization 

In [8]:
config_file = './config/config_RDF.ini'
config = configparser.ConfigParser()
config.read(config_file)

['./config/config_RDF.ini']

Then, we open the JSON histogram settings file (which location is defined in the INI file)

In [9]:
with open(config['Histogram']['bookFileName']) as f:
    histo_settings = json.loads(f.read())

For every element in the inputFiles key in the config file, add the file to the `inputFiles` list

In [10]:
inputFiles = []
for i in config['Data']['inputFiles'].split(','):
    if '.root' not in config['Data']['inputFiles']:
        if config.has_option('Data','readRedirector'):
            listdir = os.popen("xrdfs " + config['Data']['readRedirector'] + " ls " + i).read()
            inputFiles += sorted(["root://{}/{}{}".format(config['Data']['readRedirector'], i, f.split("/")[-1]) for f in listdir.split("\n") if f.endswith('.root')])  
        else:
            inputFiles += sorted([os.path.join(i, f) for f in os.listdir(i) if f.endswith('.root')])
    else:
        if config.has_option('Data','readRedirector'):
            if os.popen("xrdfs " + config['Data']['readRedirector'] + " ls " + i).read() != "":
                inputFiles.append("root://" + config['Data']['readRedirector'] + "/" + i)
        else:
            if not os.path.exists(i):
                print("\033[91m File " + i + " not Found! \033[0m")
                continue
            inputFiles.append(i)
tree_name = config['Data']['treeName']

In [11]:
print(inputFiles)  # Only one example Muon NanoAOD file

['root://eosuser.cern.ch//eos/user/t/todiotal/DTTnP_example/nano_mu_0.root']


### Push and set auxiliary files

The following cells upload the tools python module in the Dask workers (moving them in the worker's home folder)

In [12]:
config['DataFrame']['TnPTools']

'./python/TnPTools.py'

In [13]:
if distributed:
    client.register_plugin(UploadFile(config_file))
    client.register_plugin(UploadFile(config['DataFrame']['TnPTools']))

In [14]:
def set_auxiliary_files(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    os.mkdir(working_dir + '/../../../config/')
    os.mkdir(working_dir + '/../../../python/')
    shutil.copyfile(working_dir + "/" + config_file.split("/")[-1] , working_dir + '/../../../config/' + config_file.split("/")[-1])
    shutil.copyfile(working_dir + "/" + config['DataFrame']['TnPTools'].split("/")[-1] , working_dir + '/../../../python/' + config['DataFrame']['TnPTools'].split("/")[-1])
    return os.listdir("/srv/config/")

In [15]:
if distributed:
    client.run(set_auxiliary_files)

### Declare custom functions in RDF

Declare to the ROOT interpreter the header file which contains custom defined functions: this is done inside an inizialization function, which is necessary in the case of distributed execution (it is executed at the beginning of each task).

In [16]:
text_file = open(config['DataFrame']['customFunctions'], "r")
data = text_file.read()
print(config['DataFrame']['customFunctions'])

./interface/df_tools.h


In [17]:
text_file = open(config['DataFrame']['customFunctions'], "r")
data = text_file.read()
print(config['DataFrame']['customFunctions'])

def my_initialization_function():
    ROOT.gInterpreter.AddIncludePath("/usr/lib/boost_1_77_0") #/opt/conda/include
    ROOT.gInterpreter.Declare('const string configFile = "{}";'.format(config_file))
    #ROOT.gInterpreter.Declare('{}'.format(data))
    ROOT.gInterpreter.Declare(f"""
        R__LOAD_LIBRARY(libPhysics);
        {data}
    """)

if distributed:
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    my_initialization_function()

./interface/df_tools.h


### DataFrame creation and execution on Dask workers

After all the preliminary steps, we are now ready to **perform the analysis** using ROOT RDataFrame (in distributed mode).\
**The documentation for the usage of distributed RDF is available [here](https://root.cern/doc/v628/classROOT_1_1RDataFrame.html#distrdf).**

First, we set the granularity of the execution: how many partitions (i.e. tasks) we want our computation to run in. There is no fixed rule, this configuration depens on the analysis and the input file size.
In our example, we set the `npartitions` to 3 times the number of Dask workers we created in our Cluster (such information can be retrieved using the `dask.distributed` Client API).

In [18]:
if distributed:
    numWorkers= len(client.scheduler_info()['workers'])
    npartitions = 3 * numWorkers
    print("Number of workers is: {}".format(numWorkers))
    print("Number of total partitions is: {}".format(npartitions))

These are the branches we are going to use as input

In [19]:
inputBranches = ["run", "nmuon", "muon_pt", "muon_eta", "muon_phi", 
                "muon_charge", "muon_isRPC", "muon_isTracker", "muon_isTight", 
                "muon_trk_dz", "muon_pfIso04", "muon_trk_origAlgo", 
                "muon_trk_numberOfValidPixelHits", "muon_trk_numberOfValidTrackerLayers",
                "muon_rpcMu_numberOfMatchedRPCLayers", "muon_trkMu_stationMask", 
                "muon_firesIsoTrig", "muon_firesTrig", "ndtSegment", 
                "muon_matches_station", "muon_matches_wheel",
                "muon_matches_sector", "muon_matches_x", "muon_matches_y", "muon_matches_edgeX",
                "muon_matches_edgeY", "muon_matches_begin", "muon_matches_end", "dtSegment_wheel", "dtSegment_sector", "dtSegment_station",
                "dtSegment_seg4D_posLoc_x", "dtSegment_seg4D_posLoc_y", "dtSegment_seg2D_phi_t0", "dtSegment_seg4D_hasPhi", "dtSegment_seg4D_hasZed",
                "dtSegment_seg2D_phi_nHits", "dtSegment_seg2D_z_nHits"]

Initialize the dataframe (classic or distributed)

In [20]:
if distributed:
    df = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame(tree_name, inputFiles, npartitions=npartitions, daskclient=client) #len(inputFileName_list)
else:
    df = ROOT.RDataFrame(tree_name, inputFiles, inputBranches)

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


Defining the probe and pair variables (using the logic defined in the header file)

In [21]:
df = df.Define("probeIntVars", "setProbeVars<Int_t>(nmuon, muon_charge, muon_isTight, muon_isRPC, muon_firesTrig, muon_firesIsoTrig, \
               muon_isTracker, muon_trk_numberOfValidPixelHits, muon_trk_numberOfValidTrackerLayers, muon_rpcMu_numberOfMatchedRPCLayers, \
               muon_trk_origAlgo, muon_pfIso04, muon_pt, muon_eta, muon_phi, muon_trk_dz)")\
       .Define("probeFloatVars", "setProbeVars<Float_t>(nmuon, muon_charge, muon_isTight, muon_isRPC, muon_firesTrig, muon_firesIsoTrig, \
               muon_isTracker, muon_trk_numberOfValidPixelHits, muon_trk_numberOfValidTrackerLayers, muon_rpcMu_numberOfMatchedRPCLayers, \
               muon_trk_origAlgo, muon_pfIso04, muon_pt, muon_eta, muon_phi, muon_trk_dz)")\
       .Define("probeNPixelHits", "probeIntVars[0]")\
       .Define("probeNTrkLayers", "probeIntVars[1]")\
       .Define("probeNRPCLayers", "probeIntVars[2]")\
       .Define("probeOrigAlgo", "probeIntVars[3]")\
       .Define("pair_tagID", "probeIntVars[4]")\
       .Define("pair_probeID", "probeIntVars[5]")\
       .Define("probeReliso", "probeFloatVars[0]")\
       .Define("pairMass", "probeFloatVars[1]")\
       .Define("probePt", "probeFloatVars[2]")\
       .Define("pairDr", "probeFloatVars[3]")\
       .Define("pairDz", "probeFloatVars[4]")

Defining the segment efficiency variables

In [22]:
df = df.Define("segmentMapFloatEffVars", "segmentEfficiency<Float_t>(pair_probeID, muon_rpcMu_numberOfMatchedRPCLayers, run, muon_pt, muon_eta, muon_phi, muon_charge, muon_trkMu_stationMask, ndtSegment, muon_matches_station, muon_matches_wheel, muon_matches_sector, muon_matches_x, muon_matches_y, muon_matches_edgeX, muon_matches_edgeY, muon_matches_begin, muon_matches_end, dtSegment_wheel, dtSegment_sector, dtSegment_station, dtSegment_seg4D_posLoc_x, dtSegment_seg4D_posLoc_y, dtSegment_seg2D_phi_t0, dtSegment_seg4D_hasPhi, dtSegment_seg4D_hasZed, dtSegment_seg2D_phi_nHits, dtSegment_seg2D_z_nHits)")\
       .Define("segmentMapIntEffVars", "segmentEfficiency<Int_t>(pair_probeID, muon_rpcMu_numberOfMatchedRPCLayers, run, muon_pt, muon_eta, muon_phi, muon_charge, muon_trkMu_stationMask, ndtSegment, muon_matches_station, muon_matches_wheel, muon_matches_sector, muon_matches_x, muon_matches_y, muon_matches_edgeX, muon_matches_edgeY, muon_matches_begin, muon_matches_end, dtSegment_wheel, dtSegment_sector, dtSegment_station, dtSegment_seg4D_posLoc_x, dtSegment_seg4D_posLoc_y, dtSegment_seg2D_phi_t0, dtSegment_seg4D_hasPhi, dtSegment_seg4D_hasZed, dtSegment_seg2D_phi_nHits, dtSegment_seg2D_z_nHits)")\
       .Define("probeMuonEta", "segmentMapFloatEffVars[\"probeMuonEta\"]")\
       .Define("nMatchInOtherCh", "segmentMapIntEffVars[\"nMatchInOtherCh\"]")\
       .Define("effSecVsWhsecBinOnepassd", "segmentMapFloatEffVars[\"effSecVsWhAllOnesecpassd\"]")\
       .Define("effSecVsWhsecBinOnetotal", "segmentMapFloatEffVars[\"effSecVsWhAllOnesectotal\"]")\
       .Define("effSecVsWhwhBinOnepassd", "segmentMapFloatEffVars[\"effSecVsWhAllOnewhpassd\"]")\
       .Define("effSecVsWhwhBinOnetotal", "segmentMapFloatEffVars[\"effSecVsWhAllOnewhtotal\"]")\
       .Define("effSecVsWhsecBinTwopassd", "segmentMapFloatEffVars[\"effSecVsWhAllTwosecpassd\"]")\
       .Define("effSecVsWhsecBinTwototal", "segmentMapFloatEffVars[\"effSecVsWhAllTwosectotal\"]")\
       .Define("effSecVsWhwhBinTwopassd", "segmentMapFloatEffVars[\"effSecVsWhAllTwowhpassd\"]")\
       .Define("effSecVsWhwhBinTwototal", "segmentMapFloatEffVars[\"effSecVsWhAllTwowhtotal\"]")

for i in range(num_stations):
    df = df.Define("effAccVsEtaMB{}passd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsEtaMB{}passd\"]".format(str(i+1)))\
       .Define("effAccVsEtaMB{}total".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsEtaMB{}total\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaPlusMB{}etapassd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaPlusMB{}etapassd\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaPlusMB{}etatotal".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaPlusMB{}etatotal\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaPlusMB{}phipassd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaPlusMB{}phipassd\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaPlusMB{}phitotal".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaPlusMB{}phitotal\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaMinusMB{}etapassd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaMinusMB{}etapassd\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaMinusMB{}etatotal".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaMinusMB{}etatotal\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaMinusMB{}phipassd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaMinusMB{}phipassd\"]".format(str(i+1)))\
       .Define("effAccPhiVsEtaMinusMB{}phitotal".format(str(i+1)), "segmentMapFloatEffVars[\"effAccPhiVsEtaMinusMB{}phitotal\"]".format(str(i+1)))\
       .Define("effAccVsPhiPlusMB{}passd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPhiPlusMB{}passd\"]".format(str(i+1)))\
       .Define("effAccVsPhiPlusMB{}total".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPhiPlusMB{}total\"]".format(str(i+1)))\
       .Define("effAccVsPhiMinusMB{}passd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPhiMinusMB{}passd\"]".format(str(i+1)))\
       .Define("effAccVsPhiMinusMB{}total".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPhiMinusMB{}total\"]".format(str(i+1)))\
       .Define("effAccVsPtMB{}passd".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPtMB{}passd\"]".format(str(i+1)))\
       .Define("effAccVsPtMB{}total".format(str(i+1)), "segmentMapFloatEffVars[\"effAccVsPtMB{}total\"]".format(str(i+1)))\
       .Define("effVsPtMB{}passd".format(str(i+1)), "segmentMapFloatEffVars[\"effVsPtMB{}passd\"]".format(str(i+1)))\
       .Define("effVsPtMB{}total".format(str(i+1)), "segmentMapFloatEffVars[\"effVsPtMB{}total\"]".format(str(i+1)))\
       .Define("effSecVsWhMB{}secmupassd".format(str(i+1)), "segmentMapFloatEffVars[\"effSecVsWhMB{}secmupassd\"]".format(str(i+1)))\
       .Define("effSecVsWhMB{}secmutotal".format(str(i+1)), "segmentMapFloatEffVars[\"effSecVsWhMB{}secmutotal\"]".format(str(i+1)))\
       .Define("effSecVsWhMB{}whmupassd".format(str(i+1)), "segmentMapFloatEffVars[\"effSecVsWhMB{}whmupassd\"]".format(str(i+1)))\
       .Define("effSecVsWhMB{}whmutotal".format(str(i+1)), "segmentMapFloatEffVars[\"effSecVsWhMB{}whmutotal\"]".format(str(i+1)))\
       .Define("effVsNHitsPhiMB{}passd".format(str(i+1)), "segmentMapIntEffVars[\"effVsNHitsPhiMB{}passd\"]".format(str(i+1)))\
       .Define("effVsNHitsPhiMB{}total".format(str(i+1)), "segmentMapIntEffVars[\"effVsNHitsPhiMB{}total\"]".format(str(i+1)))\
       .Define("probePtMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probePtMB{}\"]".format(str(i+1)))\
       .Define("probeEtaMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probeEtaMB{}\"]".format(str(i+1)))\
       .Define("probePhiMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probePhiMB{}\"]".format(str(i+1)))\
       .Define("probePt_hasPhiMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probePt_hasPhiMB{}\"]".format(str(i+1)))\
       .Define("probeDrVsPtMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probeDrVsPtMB{}\"]".format(str(i+1)))\
       .Define("probeDxVsPtMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probeDxVsPtMB{}\"]".format(str(i+1)))\
       .Define("probeDyVsPtMB{}".format(str(i+1)), "segmentMapFloatEffVars[\"probeDyVsPtMB{}\"]".format(str(i+1)))\
       .Define("t0MB{}secBin".format(str(i+1)), "segmentMapFloatEffVars[\"t0MB{}secBin\"]".format(str(i+1)))\
       .Define("t0MB{}t0".format(str(i+1)), "segmentMapFloatEffVars[\"t0MB{}t0\"]".format(str(i+1)))

Book the histograms based on the informations provided in the JSON file.

In [23]:
booked_1Dhistos = {
    'probeNPixelHits': 'probeNPixelHits',
    'probeNTrkLayers': 'probeNTrkLayers',
    'probeNRPCLayers': 'probeNRPCLayers',
    'probeOrigAlgo': 'probeOrigAlgo',
    'probeReliso': 'probeReliso',
    'pairMass': 'pairMass',
    'pairDz': 'pairDz'}

for i in range(num_stations):
    booked_1Dhistos["effAccVsEtaMB{}".format(str(i+1))] = "effAccVsEtaMB{}".format(str(i+1))
    booked_1Dhistos["probePtMB{}".format(str(i+1))] = "probePtMB{}".format(str(i+1))
    booked_1Dhistos["probeEtaMB{}".format(str(i+1))] = "probeEtaMB{}".format(str(i+1))
    booked_1Dhistos["probePhiMB{}".format(str(i+1))] = "probePhiMB{}".format(str(i+1))
    booked_1Dhistos["effAccVsPhiPlusMB{}".format(str(i+1))] = "effAccVsPhiPlusMB{}".format(str(i+1))
    booked_1Dhistos["effAccVsPhiMinusMB{}".format(str(i+1))] = "effAccVsPhiMinusMB{}".format(str(i+1))
    booked_1Dhistos["effAccVsPtMB{}".format(str(i+1))] = "effAccVsPtMB{}".format(str(i+1))
    booked_1Dhistos["effVsPtMB{}".format(str(i+1))] = "effVsPtMB{}".format(str(i+1))
    booked_1Dhistos["effVsNHitsPhiMB{}".format(str(i+1))] = "effVsNHitsPhiMB{}".format(str(i+1))

booked_2Dhistos = {
    'probePtVsPairDr': ['probePt', 'pairDr'],
    'nOtherMatchedChVsEta': ['probeMuonEta', 'nMatchInOtherCh'],
    'effSecVsWhAllOne': ['effSecVsWhsecBinOne', 'effSecVsWhwhBinOne'],
    'effSecVsWhAllTwo': ['effSecVsWhwhBinTwo', 'effSecVsWhsecBinTwo']
}

for i in range(num_stations):
    booked_2Dhistos["effAccPhiVsEtaPlusMB{}".format(str(i+1))] = ["effAccPhiVsEtaPlusMB{}phi".format(str(i+1)), "effAccPhiVsEtaPlusMB{}eta".format(str(i+1))]
    booked_2Dhistos["effAccPhiVsEtaMinusMB{}".format(str(i+1))] = ["effAccPhiVsEtaMinusMB{}phi".format(str(i+1)), "effAccPhiVsEtaMinusMB{}eta".format(str(i+1))]
    booked_2Dhistos["probeDrVsPtMB{}".format(str(i+1))] = ["probePt_hasPhiMB{}".format(str(i+1)), "probeDrVsPtMB{}".format(str(i+1))]
    booked_2Dhistos["probeDxVsPtMB{}".format(str(i+1))] = ["probePt_hasPhiMB{}".format(str(i+1)), "probeDxVsPtMB{}".format(str(i+1))]
    booked_2Dhistos["probeDyVsPtMB{}".format(str(i+1))] = ["probePt_hasPhiMB{}".format(str(i+1)), "probeDyVsPtMB{}".format(str(i+1))]
    booked_2Dhistos["effSecVsWhMB{}".format(str(i+1))] = ["effSecVsWhMB{}secmu".format(str(i+1)), "effSecVsWhMB{}whmu".format(str(i+1))]
    booked_2Dhistos["t0MB{}".format(str(i+1))] = ["t0MB{}secBin".format(str(i+1)), "t0MB{}t0".format(str(i+1))]

booked_profile = {}

for i in range(num_stations):
    booked_profile["t0ProfileMB{}".format(str(i+1))] = ["t0MB{}secBin".format(str(i+1)), "t0MB{}t0".format(str(i+1))]

histograms = {}

TnPTools.book_1Dhistos(df, histograms, booked_1Dhistos, histo_settings)
TnPTools.book_2Dhistos(df, histograms, booked_2Dhistos, histo_settings)
TnPTools.book_profile(df, histograms, booked_profile, histo_settings)

Booked histo: probeNPixelHits
Booked histo: probeNTrkLayers
Booked histo: probeNRPCLayers
Booked histo: probeOrigAlgo
Booked histo: probeReliso
Booked histo: pairMass
Booked histo: pairDz
Booked histo: effAccVsEtaMB1
Booked histo: probePtMB1
Booked histo: probeEtaMB1
Booked histo: probePhiMB1
Booked histo: effAccVsPhiPlusMB1
Booked histo: effAccVsPhiMinusMB1
Booked histo: effAccVsPtMB1
Booked histo: effVsPtMB1
Booked histo: effVsNHitsPhiMB1
Booked histo: effAccVsEtaMB2
Booked histo: probePtMB2
Booked histo: probeEtaMB2
Booked histo: probePhiMB2
Booked histo: effAccVsPhiPlusMB2
Booked histo: effAccVsPhiMinusMB2
Booked histo: effAccVsPtMB2
Booked histo: effVsPtMB2
Booked histo: effVsNHitsPhiMB2
Booked histo: effAccVsEtaMB3
Booked histo: probePtMB3
Booked histo: probeEtaMB3
Booked histo: probePhiMB3
Booked histo: effAccVsPhiPlusMB3
Booked histo: effAccVsPhiMinusMB3
Booked histo: effAccVsPtMB3
Booked histo: effVsPtMB3
Booked histo: effVsNHitsPhiMB3
Booked histo: effAccVsEtaMB4
Booked histo

The above cells (in distributed mode) are *lazy*, meaning they don't trigger any computational graph. **The cell below**, on the other hand, **starts the execution**. You can see the resource usage in the `Workers` page of the Dask LabExtension!

In [24]:
%%time
TnPTools.write_histos(histograms, config)

Created Histo: probeNPixelHits
Created Histo: probeNTrkLayers
Created Histo: probeNRPCLayers
Created Histo: probeOrigAlgo
Created Histo: probeReliso
Created Histo: pairMass
Created Histo: pairDz
Created Histo: effAccVsEtaMB1
Created Histo: probePtMB1
Created Histo: probeEtaMB1
Created Histo: probePhiMB1
Created Histo: effAccVsPhiPlusMB1
Created Histo: effAccVsPhiMinusMB1
Created Histo: effAccVsPtMB1
Created Histo: effVsPtMB1
Created Histo: effVsNHitsPhiMB1
Created Histo: effAccVsEtaMB2
Created Histo: probePtMB2
Created Histo: probeEtaMB2
Created Histo: probePhiMB2
Created Histo: effAccVsPhiPlusMB2
Created Histo: effAccVsPhiMinusMB2
Created Histo: effAccVsPtMB2
Created Histo: effVsPtMB2
Created Histo: effVsNHitsPhiMB2
Created Histo: effAccVsEtaMB3
Created Histo: probePtMB3
Created Histo: probeEtaMB3
Created Histo: probePhiMB3
Created Histo: effAccVsPhiPlusMB3
Created Histo: effAccVsPhiMinusMB3
Created Histo: effAccVsPtMB3
Created Histo: effVsPtMB3
Created Histo: effVsNHitsPhiMB3
Created

The output is a ROOT file containing several histograms and plots. This last cell only saves a few of them as image 

In [25]:
file = ROOT.TFile.Open("./output/results.root")

pairMass = file.Get("pairMass")
canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
pairMass.Draw()
canvas.SaveAs("./output/pairMass.png")

effVsPtMB1 = file.Get("effVsPtMB1")
effVsPtMB2 = file.Get("effVsPtMB2")
effVsPtMB3 = file.Get("effVsPtMB3")
effVsPtMB4 = file.Get("effVsPtMB4")

canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
canvas.Divide(2, 2)  # 2x2 grid
canvas.cd(1)
effVsPtMB1.Draw()
canvas.cd(2)
effVsPtMB2.Draw()
canvas.cd(3)
effVsPtMB3.Draw()
canvas.cd(4)
effVsPtMB4.Draw()

canvas.SaveAs("./output/effVsPt.png")

effAccVsEtaMB1 = file.Get("effAccVsEtaMB1")
effAccVsEtaMB2 = file.Get("effAccVsEtaMB2")
effAccVsEtaMB3 = file.Get("effAccVsEtaMB3")
effAccVsEtaMB4 = file.Get("effAccVsEtaMB4")

canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
canvas.Divide(2, 2)  # 2x2 grid
canvas.cd(1)
effAccVsEtaMB1.Draw()
canvas.cd(2)
effAccVsEtaMB2.Draw()
canvas.cd(3)
effAccVsEtaMB3.Draw()
canvas.cd(4)
effAccVsEtaMB4.Draw()

canvas.SaveAs("./output/effAccVsEta.png")

Info in <TCanvas::Print>: png file ./output/pairMass.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./output/effVsPt.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./output/effAccVsEta.png has been created


Did you see the execution time, running locally (on the JLab resources)? Let's try now distributed (turning on the `distributed` flag at the beginning of the notebook!

# End of the notebook

**Additional**: move the ROOT file just created to a remote site storage (in this case the Legnaro Tier-2 storage) via Davix/https:

In [26]:
os.system("davix-put ./output/results.root " + "davs://" + config['Data']['writeRedirector'] + "/store/user/todiotal/results.root" + " -E $X509_USER_PROXY --capath $X509_CERT_DIR")

0